In [61]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import re

In [3]:
businesses= pd.read_csv("/project/business_data.csv")
predestrian_crossings= pd.read_csv("/project/pedestrian_crossings.csv")
predestrian_traffic= pd.read_csv("/project/pedestrian_traffic.csv")
street= pd.read_csv("/project/street_data.csv")
walkability= pd.read_csv("/project/walkability.csv")

In [20]:
businesses.shape
businesses.head(5)


(999, 29)

,ID:,Timestamp,Storefront,Visible Sign,WiFi,Tables inside / Tables on sidewalk,Plugs,CCTV,External CCTV,Number of female employees,Accepts credit card,Name of Business,Unnamed: 12,Type of Business,Coordinates,Open- Mon,Open- Tues,Open- Wed,Open-Thursday,Open-Friday,Open-Saturday,Open- Sunday,Time Open,Time Close,Opens a different time,Pictures,Business Owner,District,Neighborhood
0,SHANTI001,4/15/2022 16:38:51,Si,Si,Si,Si,Si,Si,Si,3.0,NaN,Carnes asadas el rancho,NaN,Food joint,"19.440448735198807, -99.11258785714946",Si,Si,Si,Si,Si,Si,Si,2:00:00 pm,2:00:00 am,Viernes sábado y domingo abré de 2 pm a 3 pm,https://drive.google.com/open?id=1QAjuqIMLKoPc...,Lucio Mora Gálvez,Venustiano Carranza,20 de nov
1,SHANTI002,4/15/2022 17:14:53,Si,Si,Si,Si,Si,Si,Si,2.0,NaN,Chocoland,NaN,Restaurant,"19.4435967, -99.0978994",Si,Si,Si,Si,Si,Si,Si,9:00:00 am,11:00:00 am,NaN,https://drive.google.com/open?id=116t9fuxx56Cv...,Andrea Barrera,Venustiano Carranza,Romero Rubio
2,SHANTI003,4/20/2022 12:14:13,Si,Si,Si,Si,Si,Si,Si,0.0,NaN,Sabor México,NaN,Restaurant,"19.441063374431312, -99.13819952984495",Si,Si,Si,Si,Si,Si,Si,6:00:00 am,6:00:00 am,Las 24 hrs está abierto,https://drive.google.com/open?id=1iWdSm-IcP19L...,Carlos Romero Baron,Cuauhtémoc,Centro
3,SHANTI004,4/20/2022 12:36:29,Si,Si,No,Si,Si,Si,Si,4.0,NaN,Los Monchis,NaN,Restaurant,"19.440604101437213, -99.13770269306984",Si,Si,Si,No,Si,Si,Si,9:00:00 am,7:00:00 pm,Los jueves descansan,https://drive.google.com/open?id=1t4ELC3ew5yBq...,Diana Morros Lozada,Cuauhtémoc,Centro
4,SHANTI005,4/20/2022 14:14:39,Si,Si,Si,Si,Si,Si,Si,0.0,NaN,Gallo Negro,NaN,Restaurant,"19.433174357938398, -99.13937294323891",Si,Si,Si,Si,Si,Si,Si,12:00:00 pm,11:00:00 pm,NaN,https://drive.google.com/open?id=1Qp41flGJFGMz...,Daniel Escalona Recendiz,Cuauhtémoc,Centro


In [22]:
# Count NaN values for each column
businesses_nan_count = businesses.isna().sum()

# Print the count of NaN values for each column
print(businesses_nan_count)

ID:                                   101
Timestamp                             101
Storefront                            101
Visible Sign                          101
WiFi                                  101
Tables inside / Tables on sidewalk    101
Plugs                                 101
CCTV                                  101
External CCTV                         101
Number of female employees            106
Accepts credit card                   889
Name of Business                      101
Unnamed: 12                           999
Type of Business                      101
Coordinates                           101
Open- Mon                             113
Open- Tues                            111
Open- Wed                             106
Open-Thursday                         103
Open-Friday                           101
Open-Saturday                         114
Open- Sunday                          143
Time Open                             101
Time Close                        

In [28]:
# Custom function to convert coordinates to float if possible, else return NaN
def convert_to_float(coord):
    if coord is None:
        return float('NaN')
    try:
        return float(coord)
    except ValueError:
        return float('NaN')

In [35]:
# Split the 'Coordinates' column into a temporary DataFrame
temp_df = businesses['Coordinates'].str.split(',', expand=True)
# Drop the unnecessary columns
temp_df = temp_df.drop(columns=[2, 3])
# Apply the custom function to 'Latitude' and 'Longitude' columns
temp_df[0] = temp_df[0].apply(convert_to_float)
temp_df[1] = temp_df[1].apply(convert_to_float)

# Rename the temporary DataFrame columns and concatenate it with the original DataFrame
temp_df.columns = ['Latitude', 'Longitude']
businesses = pd.concat([businesses, temp_df], axis=1)

# Drop rows containing NaN in 'Latitude' and 'Longitude' columns
businesses.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Drop the 'Coordinates' column (optional)
businesses.drop('Coordinates', axis=1, inplace=True)



KeyError: 'Coordinates'

In [52]:
# Print the DataFrame with the new columns
businesses.head(1)

,ID:,Timestamp,Storefront,Visible Sign,WiFi,Tables inside / Tables on sidewalk,Plugs,CCTV,External CCTV,Number of female employees,Accepts credit card,Name of Business,Type of Business,Open- Mon,Open- Tues,Open- Wed,Open-Thursday,Open-Friday,Open-Saturday,Open- Sunday,Time Open,Time Close,Opens a different time,Pictures,Business Owner,District,Neighborhood,Latitude,Longitude
0,SHANTI001,4/15/2022 16:38:51,1,1,1,1,1,1,1,3.0,NaN,Carnes asadas el rancho,Food joint,1,1.0,1.0,1.0,1,1,1,2:00:00 pm,2:00:00 am,Viernes sábado y domingo abré de 2 pm a 3 pm,https://drive.google.com/open?id=1QAjuqIMLKoPc...,Lucio Mora Gálvez,Venustiano Carranza,20 de nov,19.440449,-99.112588


In [38]:
# Drop an empty column created when converting from Excel
businesses.drop('Unnamed: 12', axis=1, inplace=True)

array(['Si', 'No'], dtype=object)

array(['Si', 'No', 'Si, No'], dtype=object)

array(['Si', 'No'], dtype=object)

array(['Si', 'No'], dtype=object)

array(['Si', 'No'], dtype=object)

KeyError: 'CCTV'

Replace "Si" with 1 and "No" with 0

In [50]:
# List of columns to update
columns_to_update = ['Storefront', 'Visible Sign', 'WiFi',
       'Tables inside / Tables on sidewalk', 'Plugs', 'CCTV ', 'External CCTV', 'Open- Mon', 'Open- Tues', 'Open- Wed',
       'Open-Thursday', 'Open-Friday', 'Open-Saturday', 'Open- Sunday']

# Replace 'Si' with 1 and 'No' with 0 in the specified columns
for column in columns_to_update:
    businesses[column] = businesses[column].replace({'Si': 1, 'No': 0})

In [63]:
predestrian_crossings.shape
predestrian_crossings.head(1)

(199, 16)

,ID,Remarks,Time,Incline,Width,Ramp conditions,Ramp // Crosswalk\n,Ramp obstacles,Crosswalk\n,Crosswalk width\n,Vertical signs,Crossing length,Access to people with disabilities,Score,Longitude,Latitude
0,394,NaN,12/05/2022 15:06:49.394 CST,1 (maxima 6%),0 (menos de 1.2m),1 (superficie plana),1 (rampa alineada),1 (ningun obstaculo ni impedimento),1 (completamente pintados),1 (mas de 3m),0 (no existe señal),1 (3 m en un carril/6 m en dos carriles),0 (no hay pavimento podotactil ni semaforo aud...,7.0,-99.136208,19.434242


In [58]:
predestrian_crossings.drop(['Unnamed: 17', 'Unnamed: 18','Unnamed: 19','Unnamed: 20',
                           'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
                           'Unnamed: 25',"Latitude", "Longitude"], axis=1, inplace=True)

In [62]:
# Extract the coordinates from the 'Coordinates' column using regex
pattern = re.compile(r"(-?\d+\.\d+)")
temp1_df = predestrian_crossings['Coordinates'].apply(lambda x: pattern.findall(x))

# Create a new DataFrame from the extracted coordinates
temp1_df = pd.DataFrame(temp1_df.to_list(), columns=['Longitude', 'Latitude', 'Altitude'])

# Drop the 'Altitude' column
temp1_df.drop('Altitude', axis=1, inplace=True)

# Apply the custom function to 'Latitude' and 'Longitude' columns
temp1_df['Latitude'] = temp1_df['Latitude'].apply(convert_to_float)
temp1_df['Longitude'] = temp1_df['Longitude'].apply(convert_to_float)

# Concatenate the temporary DataFrame with the original DataFrame
predestrian_crossings = pd.concat([predestrian_crossings, temp1_df], axis=1)

# Drop the 'Coordinates' column 
predestrian_crossings.drop('Coordinates', axis=1, inplace=True)

In [68]:
predestrian_traffic.shape
predestrian_traffic.head(1)

(11, 4)

,ID,Score,Latitude,Longitude
0,470,B(16-23),19.427375,-99.133563


In [72]:
def extract_latitude_longitude(coordinate_str):
    if isinstance(coordinate_str, str):
        try:
            match = re.search(r'\(([^)]+)', coordinate_str)
            if match:
                coords = match.group(1).split()
                return float(coords[1]), float(coords[0])
            else:
                return float('NaN'), float('NaN')
        except ValueError:
            return float('NaN'), float('NaN')
    else:
        return float('NaN'), float('NaN')

In [65]:
predestrian_traffic = predestrian_traffic[['ID', 'Coordinates', 'Score']]
# Apply the custom function to extract latitude and longitude
predestrian_traffic['Latitude'], predestrian_traffic['Longitude'] = zip(*predestrian_traffic['Coordinates'].apply(extract_latitude_longitude))
# Drop the 'Coordinates' column
predestrian_traffic.drop('Coordinates', axis=1, inplace=True)

In [74]:
street.shape
street.head(1)

(999, 9)

,ID,Date,Hour,Variable,Classification,Description,Picture,Latitude,Longitude
0,1.0,11/22/2022,14:48:26.758,Physical aspects in poor condition,Trash on the street and facade in poor condition,NaN,recolección de datos - centro_20221122_1448595...,19.435848,-99.148059


In [69]:
street = street[['ID', 'Coordinates', 'Date', 'Hour', 'Variable', 'Classification', 'Description',
                'Picture']]

In [73]:
# Apply the custom function to extract latitude and longitude
street['Latitude'], street['Longitude'] = zip(*street['Coordinates'].apply(extract_latitude_longitude))
# Drop the 'Coordinates' column
street.drop('Coordinates', axis=1, inplace=True)

In [78]:
walkability.shape
walkability.head(1)

(122, 20)

,ID,Time,Latitude,Longitude,Pedestrian strip width,Strip services,Material,Status of maintenance,Continuity,Mobiles obstacles,Permanent obstacles,Vertical Obstacles,Street furniture,Trees,Planters,Street lighting\n,Facades activity,Percentage of parking accesses,visibilidad de fachada,Score
0,427,12/05/2022 15:51:39.329 CST,19.437174,-99.136807,1 (mas de 1.8m),1 (si existe),1 (material que no resbale),1 (sin grietas o levantamientos),0 (hay desniveles),1 (no hay obstaculos),1 (no hay obstaculos),1 (no hay obstaculos),1 (una o mas bancas),0.5 (por lo menos la mitad de la acera tiene a...,0.5 (la mitad tiene jardineras),1 (hay postes que miran a la acera en todo el ...,1 (al menos un local o comercio),1 (no existe o uno ocupa menos del 30% del lote),1 (la mayoria es visible),13.0


In [75]:
walkability.drop(['Unnamed: 1','Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
                           'Unnamed: 25'], axis=1, inplace=True)

In [77]:
# Apply the custom function to extract latitude and longitude
walkability['Latitude'], walkability['Longitude'] = zip(*walkability['Coordinates'].apply(extract_latitude_longitude))
# Drop the 'Coordinates' column
walkability.drop('Coordinates', axis=1, inplace=True)